In [15]:
import numpy as np
from netCDF4 import Dataset
import glob
import matplotlib.pyplot as plt
import scipy.stats as sp
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error,precision_score, accuracy_score, confusion_matrix, classification_report, recall_score, f1_score
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, FixedLocator)
from math import cos, asin, sqrt, pi

import warnings
warnings.filterwarnings('ignore')
import sys

from sklearn.ensemble import RandomForestRegressor
clf_RF = RandomForestRegressor(max_depth=400, random_state=0)

In [2]:
init="May" # forecast "initialisation" month
trg="MJJ" # forecast target month

dataset=Dataset("past2k_tasmax_HWs_EUR_MJJA_period70018850_clim88218850.nc",'r')
lons=dataset['lon'][:]
lats=dataset['lat'][:]

In [3]:
# Solutions files for each grid point #
files=sorted(glob.glob("opt_MayMJJ_past2k_*.csv"))
print ("Number of grid points: ",len(files))

Number of grid points:  1


In [4]:
### Open solutions and extract best solution ###

# Lists for best solutions #
xs=[] # x-coord
ys=[] # y-coord
nevals=[] # number of evaluations 
cv_best=[] # best cross-validation/training score
test_best=[] # test score corresponding to cv_best
sols_best=[] # predictors correspondin to cv_best

for file in files:
    #print (file[-9:-4])

    label=file[-9:-4].split("_") # identify x-coord and y-coord in file name

    sol_file_av = pd.read_csv(file, index_col=None, sep=' ', header=0)#[:20]
    if sol_file_av.shape[0]>0:
        xs.append(int(label[1]))
        ys.append(int(label[0]))
        nevals.append(sol_file_av.shape[0])
        sols_best.append(np.fromstring(sol_file_av.Sol[sol_file_av.sort_values(by=['CV'],ascending=True).index[0]].replace('[', '').replace(']', '').replace('\n', ''), dtype=float, sep=' '))
        cv_best.append(sol_file_av.CV[sol_file_av.sort_values(by=['CV'],ascending=True).index[0]])
        test_best.append(sol_file_av.Test[sol_file_av.sort_values(by=['CV'],ascending=True).index[0]])
    else:
        print ("Empty file - no solutions")


In [5]:
# Mapping of past2k to higher resolution ERA5
#target_2D=np.concatenate((target_past2k,target_ERA5))

# 1 - Input ERA5 lon and lat
# 2 - Find nearest past2k point
# 3 - Identify p value
# 4 - Concatenate past2k and ERA5 HWs

def distance(lat1, lon1, lat2, lon2):
    r = 6371 # km
    p = pi / 180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 2 * r * asin(sqrt(a))

def find_nearest(era_lat,era_lon):
    dmin=9E15
    dmin_ind=0
    #print (era_lat,era_lon)
    for x in range(len(xs)):
        #print (lats[ys[x]],lons[xs[x]])
        d=distance(era_lat,era_lon,lats[ys[x]],lons[xs[x]])
        if d<dmin:
            dmin=d
            dmin_ind=x
        #print (d)
    return dmin_ind

In [6]:
### Reformat optimal prpedictors ###

def create_board(n_rows, n_cols, final_sequence, sequence_length, feat_sel):
    board = np.zeros((n_rows, n_cols))
    
    for i in range(n_cols):
        start_index = int(final_sequence[i]) 
        end_index = int(final_sequence[i])  + int(sequence_length[i])
        if feat_sel[i] != 0:
            board[start_index:end_index, i] = 1
    
    return board

In [7]:
### Produce forecasts based on optimal predictors ###

# Inputs: p - grid point index, mod - ML model, pred_dataframe - full list of predcitors, remove-co2 - boolean to remove CO2 as predictor
def forecast(p,mod,pred_dataframe,remove_co2):
    fn=find_nearest(lats_era5_tile[p],lons_era5_tile[p])

    target=np.concatenate((target_past2k[:,ys[fn],xs[fn]],target_ERA5.reshape(l,target_ERA5[0].size)[:,p]))

    array_best=sols_best[fn]

    #array_best=sols_best[p]
    #target=target_2D[:,ys[p],xs[p]]
    
    df=pd.DataFrame(target,columns=['Target'])
    df.index = target_dates
    target_dataset=df

    split=int(array_best.shape[0]/3)
    
    sequence_length_best = array_best[0:split]
    final_sequence_best = array_best[split:split*2]
    feat_sel_best = array_best[split*2:]

    if remove_co2:
        sequence_length_best=np.delete(sequence_length_best, -2)
        final_sequence_best=np.delete(final_sequence_best, -2)
        feat_sel_best=np.delete(feat_sel_best, -2)
        n_cols = split-1
    else:
        n_cols = split
    n_rows = int((sequence_length_best + final_sequence_best).max())+10
    
    board_best = create_board(n_rows, n_cols, final_sequence_best, sequence_length_best, feat_sel_best)

    time_lags = np.repeat(0,pred_dataframe.shape[1])
    time_sequences = np.repeat(n_rows,n_cols)
    variable_selection = np.repeat(1,pred_dataframe.shape[1])
    
    dataset_opt = target_dataset.copy()
    for i,col in enumerate(pred_dataframe.columns):      
        if remove_co2:
            if col=="data_CO2":
                continue
        if col=="weekofyear":
            i=i-1
        if variable_selection[i] == 0:
            continue
        for j in range(time_sequences[i]):
            dataset_opt[str(col)+'_lag'+str(time_lags[i]+j)] = pred_dataframe[col].shift(time_lags[i]+j)

        
    first_train_index=int(np.argwhere(df.index==first_train))
    last_train_index=int(np.argwhere(df.index==last_train))
    
    train_dataset_opt = dataset_opt[first_train_index:last_train_index]
    test_dataset_opt = dataset_opt[last_train_index:]

    Y_column = 'Target' 
           
    X_train=train_dataset_opt[train_dataset_opt.columns.drop([Y_column]) ]
    Y_train=train_dataset_opt[Y_column]
    X_test=test_dataset_opt[test_dataset_opt.columns.drop([Y_column]) ]
    Y_test=test_dataset_opt[Y_column]

    from sklearn import preprocessing
    scaler = preprocessing.StandardScaler()
    X_std_train = scaler.fit(X_train)

    X_std_train = scaler.transform(X_train)
    X_std_test = scaler.transform(X_test)

    X_train=pd.DataFrame(X_std_train,columns=X_train.columns,index=X_train.index)
    X_test=pd.DataFrame(X_std_test,columns=X_test.columns,index=X_test.index)
    
    X_train_new = np.array(X_train)[:,board_best.T.reshape(1,-1)[0].astype(bool)]
    X_test_new = np.array(X_test)[:,board_best.T.reshape(1,-1)[0].astype(bool)]

    clf = mod
    clf.fit(X_train_new, Y_train)
    predictions = clf.predict(X_test_new)
    Y_pred = pd.DataFrame(predictions, columns=['Y_pred'], index=Y_test.index)
    
    return clf.__class__.__name__,predictions # full model name, 1D output forecast (year)

In [8]:
# Loop "forecast" function through all points ###
# Input: clf - model, l - length in years of test period
def predout(clf,l,pred_dataframe,remove_co2):
    preds=np.zeros((len(xs),l)) #(point, year)

    for point in range(len(xs)):
        #print (point)
        output=forecast(point,clf,pred_dataframe,remove_co2)
        preds[point]=output[1]
        
    return preds

In [21]:
### Save output of forecast/predout ###
def saver(preds,mod,l):
    fout=Dataset("Forecasts_initMay_targMJJ_{2}_19792021.nc".format(init,trg,mod), "w")

    lat = fout.createDimension('x', len(xs))
    day = fout.createDimension('year', l) 

    var = fout.createVariable('lat', int, ('x')) 
    var.standard_name = "Latitude"
    var[:]=ys
    var = fout.createVariable('lon', int, ('x')) 
    var.standard_name = "Longitude"
    var[:]=xs

    var = fout.createVariable('pred', int, ('x','year')) 
    var.standard_name = "NDQ90 predictions"
    var[:]=np.array(preds)

    fout.close()

In [18]:
### Open HW target dataset ###

dataset=Dataset("past2k_tasmax_HWs_EUR_MJJA_period70018850_clim88218850.nc",'r')
target_past2k=dataset['NDQ90_May'][:,0,:,:]+dataset['NDQ90_Jun'][:,0,:,:]+dataset['NDQ90_Jul'][:,0,:,:]

dataset=Dataset("ERA5_tmax_HWs_EUR_NDQ90_period19402022_clim19932016.nc",'r') 
target_ERA5=dataset['NDQ90_May'][1979-1940:2021-1940,0,:,:]+dataset['NDQ90_Jun'][1979-1940:2021-1940,0,:,:]+dataset['NDQ90_Jul'][1979-1940:2021-1940,0,:,:]
lons_ERA5=dataset['lon'][:]
lats_ERA5=dataset['lat'][:]

lons_era5_tile=np.tile(lons_ERA5,lats_ERA5.size)
lats_era5_tile=np.tile(lats_ERA5,lons_ERA5.size)

#target_2D=np.concatenate((target_past2k,target_ERA5))
   
target_dates=[] # dummy date for summer HWMI
train_years_past2k=range(7001,8851,1)
train_years_era5=range(1979,2021,1)
for year in train_years_past2k:
    target_dates.append(str(year).zfill(4)+"-04-30") # Days in June #
for year in train_years_era5:
    target_dates.append(str(year).zfill(4)+"-04-30") # Days in June #

#===============================#

first_train = "7002-04-30"
last_train = "1979-04-30"

pred_dataframe_era5 = pd.read_csv('Predictors_dataset_ERA5_weekly.csv', index_col=0)

pred_dataframe_past2k = pd.read_csv('Predictors_dataset_past2k_weekly.csv', index_col=0)

pred_dataframe=pd.concat([pred_dataframe_past2k,pred_dataframe_era5])

# Convert ERA5 predictor to past2k units
# Soil Moisture kg/m2 , ERA5 - m3/s3 (divide by 0.1m, divide by 1000 kg.m3, times by 0.7 = divide by 70)
pred_dataframe['smEurope_cluster1']['1979-01-01':]=(pred_dataframe['smEurope_cluster1']['1979-01-01':].values)*70
pred_dataframe['smEurope_cluster2']['1979-01-01':]=(pred_dataframe['smEurope_cluster2']['1979-01-01':].values)*70
pred_dataframe['smEurope_cluster3']['1979-01-01':]=(pred_dataframe['smEurope_cluster3']['1979-01-01':].values)*70
pred_dataframe['smEurope_cluster4']['1979-01-01':]=(pred_dataframe['smEurope_cluster4']['1979-01-01':].values)*70
pred_dataframe['smEurope_cluster5']['1979-01-01':]=(pred_dataframe['smEurope_cluster5']['1979-01-01':].values)*70

# SIC Arctic
# past2k - percentage , ERA5 - proportion 
pred_dataframe['sicArctic_cluster1']['1979-01-01':]=pred_dataframe['sicArctic_cluster1']['1979-01-01':].values*100
pred_dataframe['sicArctic_cluster2']['1979-01-01':]=pred_dataframe['sicArctic_cluster2']['1979-01-01':].values*100
pred_dataframe['sicArctic_cluster3']['1979-01-01':]=pred_dataframe['sicArctic_cluster3']['1979-01-01':].values*100
pred_dataframe['sicArctic_cluster4']['1979-01-01':]=pred_dataframe['sicArctic_cluster4']['1979-01-01':].values*100
pred_dataframe['sicArctic_cluster5']['1979-01-01':]=pred_dataframe['sicArctic_cluster5']['1979-01-01':].values*100

l=42 # (length of period: 1950-2022)

In [22]:
remove_co2=True

preds=predout(clf_RF,l,pred_dataframe,remove_co2)
saver(preds,"RF",l)